In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [33]:
# Data
train_csv = '/content/sign_mnist_train.csv'
test_csv = '/content/sign_mnist_test.csv'

categories = 24

# Settings
input_tensor = tf.keras.Input(shape=(28, 28, 3))

# Train settings
batch_size=128 # was 32
epochs=50

### Load Data

In [34]:
# Load with pandas (more simple)
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [35]:
# Select labels
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

In [36]:
# Select features
x_train = train_df.values
x_test = test_df.values

In [37]:
# Reshape features to images
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [38]:
x_train = np.concatenate([x_train] * 3, axis=-1)
x_test = np.concatenate([x_test] * 3, axis=-1)

In [39]:
print(x_train.shape)
print(x_test.shape)

(27455, 28, 28, 3)
(7172, 28, 28, 3)


### Pre-process data

In [40]:
from sklearn.preprocessing import LabelBinarizer

# Put target col in categorical format
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

In [41]:
def preprocess_data(image):
  image_p = tf.keras.applications.resnet50.preprocess_input(image)
  return image_p

In [42]:
x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

### Data augmentation preparation

In [43]:
from keras.preprocessing.image import ImageDataGenerator

#Data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False,
        validation_split=0.2)

datagen.fit(x_train)

### Learning rate improvements

In [44]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

### Load Model

In [45]:
feature_extractor = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=input_tensor
)

In [46]:
# Freeze layers exept last block
for layer in feature_extractor.layers[:143]:
  layer.trainable = False

### Make Model

In [47]:
model = tf.keras.models.Sequential()

In [48]:
# Feature extraction
model.add(feature_extractor)

# Classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(categories, activation='softmax'))

### Compile and Fit

In [49]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 24)                49176     
                                                                 
Total params: 23636888 (90.17 MB)
Trainable params: 15025176 (57.32 MB)
Non-trainable params: 8611712 (32.85 MB)
_________________________________________________________________


In [50]:
save_model = tf.keras.callbacks.ModelCheckpoint(
    filepath='model.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
run_hist = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size,
                                  subset='training'),
                     validation_data = datagen.flow(x_train, y_train,
                                                  batch_size=batch_size,
                                                  subset='validation'),
                     epochs = epochs,
                     callbacks=[save_model, learning_rate_reduction])

Epoch 1/50
172/172 [==============================] - ETA: 0s - loss: 2.3305 - accuracy: 0.3742
Epoch 1: val_accuracy improved from -inf to 0.49007, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


172/172 [==============================] - 704s 4s/step - loss: 2.3305 - accuracy: 0.3742 - val_loss: 1.8926 - val_accuracy: 0.4901 - lr: 2.0000e-05
Epoch 2/50
168/172 [============================>.] - ETA: 15s - loss: 0.9622 - accuracy: 0.7202

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy and loss
plt.plot(run_hist.history['accuracy'], label='train')
plt.plot(run_hist.history['val_accuracy'], label='test')
plt.plot(run_hist.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_test, y_pred)
print(f"Kappa coefficient : {kappa}")